In [39]:
# Importing packages

from math import *
from numpy import *
from PIL import Image, ImageDraw
from csv import reader
import csv
from os import listdir
from os.path import isfile, join

In [42]:
# Choosing Front (True) or Back (False) Camera

is_frontCam = False # /!\ modify this value according to your choice of camera /!\
image_folder_name = (lambda x: 'FrameFrontCam' if is_frontCam else 'FrameBackCam')(0)

In [43]:
# Loading the files with the LiDAR data points

csv.field_size_limit(9999999)

base_path = r"enter\your\base\path\data"
file_paths = {'IBEO': base_path + r"\IBEO\IBEO_Scan_XYZ_Points.csv",
              'Velodyne': base_path + r"\Velodyne\VLP16_Scan_XYZRGB_Points.csv",
              'Camera directory': base_path + r"\Camera" + '\\' + image_folder_name}
# new_path = ... # add the path of the directory in which you want to save the images enriched by LiDAR data

with open(file_paths['IBEO'], 'r') as read_obj:
    csv_reader = reader(read_obj)
    rows_IBEO = list(csv_reader)

with open(file_paths['Velodyne'], 'r') as read_obj:
    csv_reader = reader(read_obj)
    rows_VLP = list(csv_reader)

In [44]:
# Loading the images from the camera and sorting them by alphanumeric order

image_path = file_paths['Camera directory']
image_list = [im for im in listdir(image_path) if isfile(join(image_path, im))]
image_list.sort()

In [45]:
# Defining parameters

if is_frontCam:
    phi = 1.5 # pan angle of the camera around the y-axis (in degrees)
    psi = 0 # tilt angle of the camera around the x-axis (in degrees)
    theta = -2.5 # roll angle of the camera around the z-axis (in degrees)
    # Translation vector (IBEO -> front camera)
    transl_IBEO_cam = [1.932, 0.25, 1.4] # in meters
    # Seconds in the video
    seconds = linspace(0, 93, 193)
else:
    phi = 1.5 # pan angle of the camera around the y-axis (in degrees)
    psi = 0 # tilt angle of the camera around the x-axis (in degrees)
    theta = 177.5 # roll angle of the camera around the z-axis (in degrees)
    # Translation vector (IBEO -> back camera)
    transl_IBEO_cam = [0, 0.25, 1.4] # in meters
    # Seconds in the video
    seconds = linspace(0, 91, 186)

f = 0.012 # focal of the camera in meters
l_sensor = 0.01125 # in meters
h_sensor = 0.00713 # in meters
size_pixel = 5.86 * 10**(-6) # in meters
init_instant = 605179 # first recorded time stamp

# Translation vector (IBEO -> Velodyne)
transl_IBEO_VLP = [0.677, 0, 1.487]  # in meters

In [46]:
# Defining auxiliary functions

def get_row(rows, instant): # instant (in seconds) in the video
    """
    Returns the row with the spatial data points given the time (instant, in seconds) in the video.
    """
    t = instant * 1000000 + init_instant
    for row in rows:
        row = list(row[0].split(";"))
        if int(row[0]) > t:
            return row

def rot_x(psi):
    """
    Returns the rotation matrix of angle psi (in degrees) around the x-axis.
    """
    psi_ = radians(psi) # we need to convert psi to radians before using trigonometric functions
    return array([[1, 0, 0], [0, cos(psi_), -sin(psi_)], [0, sin(psi_), cos(psi_)]])

def rot_y(phi):
    """
    Returns the rotation matrix of angle phi (in degrees) around the y-axis.
    """
    phi_ = radians(phi)
    return array([[cos(phi_), 0, sin(phi_)], [0, 1, 0], [-sin(phi_), 0, cos(phi_)]])

def rot_z(theta):
    """
    Returns the rotation matrix of angle theta (in degrees) around the z-axis.
    """
    theta_ = radians(theta)
    return array([[cos(theta_), -sin(theta_), 0], [sin(theta_), cos(theta_), 0], [0, 0, 1]])

def from_lidar_to_camera(X_lidar, is_VLP):
    """
    Input (list): [x, y, z] coordinates in the VLP reference frame
    Output (list): [x, y, z] coordinates in the camera reference frame
    """

    X_lidar = array(X_lidar)
    t_IBEO_VLP = array(transl_IBEO_VLP) * is_VLP
    t_IBEO_cam = array(transl_IBEO_cam)

    X_ = t_IBEO_VLP - t_IBEO_cam + X_lidar

    X_cam = dot(rot_z(theta), dot(rot_y(phi), dot(rot_x(psi), X_)))
    
    return list(X_cam)

def coord_lim(x):
    """
    Returns the (y_limit, z_limit) values to see the object in the image, in the camera reference frame.
    """
    y_lim = (l_sensor / 2) * x / f
    z_lim = (h_sensor / 2) * x / f
    return y_lim, z_lim

def coord_image(x, y, z):
    """
    Returns the (y_image, z_image) coordinates in the sensor for an object of coordinates (x, y, z) in the camera
    reference frame.
    """
    y_im = y * f / x
    z_im = z * f / x
    return y_im, z_im

def coord_pixel(y_im, z_im):
    """
    Returns the (y_pixel, z_pixel) coordinates in the image for an object of coordinates (y_im, z_im) in the sensor.
    """
    y_pix = int(y_im / size_pixel)
    z_pix = int(z_im / size_pixel)
    return y_pix, z_pix

def n_pixel(x, y, z):
    y_lim, z_lim = coord_lim(x)
    y_im, z_im = coord_image(x, y, z)
    if abs(y) < y_lim and abs(z) < z_lim:
        y_pix, z_pix = coord_pixel(y_im, z_im)
        return [x, y_pix, z_pix]
    return None

def from_lidar_to_pixel(X_lidar, is_VLP):
    x_cam, y_cam, z_cam = from_lidar_to_camera(X_lidar, is_VLP)
    return n_pixel(x_cam, -y_cam, -z_cam)

def rgb(mini, maxi, value, transparency):
    mini, maxi = float(mini), float(maxi)
    ratio = 2 * (value - mini) / (maxi - mini)
    r = int(max(0, 255 * (1 - ratio)))
    b = int(max(0, 255 * (ratio - 1)))
    g = 255 - b - r
    return r, g, b, transparency

In [49]:
def main():
    """
    Generates the images enriched with the LiDAR data points at each frame.
    """

    for i, sec in enumerate(seconds):

        data_points_list_VLP = [float(x) for x in get_row(rows_VLP, sec)]
        x_list_VLP = [data_points_list_VLP[k] for k in range(len(data_points_list_VLP)) if k % 6 == 1]
        y_list_VLP = [data_points_list_VLP[k] for k in range(len(data_points_list_VLP)) if k % 6 == 2]
        z_list_VLP = [data_points_list_VLP[k] for k in range(len(data_points_list_VLP)) if k % 6 == 3]

        data_points_list_IBEO = [float(x) for x in get_row(rows_IBEO, sec)]
        x_list_IBEO = [data_points_list_IBEO[k] for k in range(len(data_points_list_IBEO)) if k % 3 == 1]
        y_list_IBEO = [data_points_list_IBEO[k] for k in range(len(data_points_list_IBEO)) if k % 3 == 2]
        z_list_IBEO = [data_points_list_IBEO[k] for k in range(1, len(data_points_list_IBEO)) if k % 3 == 0]

        x_list = x_list_VLP + x_list_IBEO
        y_list = y_list_VLP + y_list_IBEO
        z_list = z_list_VLP + z_list_IBEO

        # Still not making use of the R, G & B values given to represent the reflexion intensity (VLP only)
        # r_list = [data_points_list_VLP[k] for k in range(len(data_points_list)) if k % 6 == 4]
        # g_list = [data_points_list_VLP[k] for k in range(len(data_points_list)) if k % 6 == 5]
        # b_list = [data_points_list_VLP[k] for k in range(1, len(data_points_list)) if k % 6 == 0]

        lidar_points = []
        for j in range(len(x_list)):
            lidar_points.append([x_list[j], y_list[j], z_list[j]])

        lidar_pixels = []
        lidar_points_kept = [] # list of data points kept in lidar_pixels (but in meters)
        for k, vec in enumerate(lidar_points):
            is_VLP = (k < len(x_list_VLP))

            if from_lidar_to_pixel(vec, is_VLP):
                pixel_position = from_lidar_to_pixel(vec, is_VLP)
                real_position = from_lidar_to_camera(vec, is_VLP)
                lidar_pixels.append(pixel_position)
                lidar_points_kept.append(real_position)

        # Loading image from directory
        image_file_name = image_list[i]
        image_path_name = image_path + '\\' + image_file_name
        input_image = Image.open(image_path_name)

        width, height = input_image.size
        draw = ImageDraw.Draw(input_image, 'RGBA')

        for l, data_point in enumerate(lidar_pixels):
            x, y, z = data_point
            y += 960
            z += 608

            x_r, y_r, z_r = lidar_points_kept[l]

            D = sqrt(x_r**2 + y_r**2 + z_r**2) # distance between data point and camera

            R_D_eff = (100 - D) / (5 * sqrt(2))

            y_upperleft = int(max(0, y - R_D_eff))
            z_upperleft = int(max(0, z - R_D_eff))
            y_lowerright = int(min(width, y + R_D_eff))
            z_lowerright = int(min(height, z + R_D_eff))

            position = (y_upperleft, z_upperleft, y_lowerright, z_lowerright)

            draw.ellipse(position, fill=rgb(0, 100, D, 75))

        # input_image.save(new_path) # uncomment to save images enriched by IBEO and Velodyne data
        print(image_file_name)
        input_image.show()

In [50]:
if __name__ == '__main__':
    main()

frame00001.png
